In [6]:
import requests
VICUNA_URL = 'http://10.100.100.104:8013/v1/completions'
VICUNA_HEADER = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}
prompt = 'Write me a python script that print "Hello World"'
json_data = {
            "model": "vicuna-13b-v1.5-16k",
            "max_tokens": 256,
            'prompt': prompt,
            "temperature": 0.0,
            "frequency_penalty": 0.0,
            "presence_penalty": 0.0,
        }

response = requests.post(
            VICUNA_URL,
            headers=VICUNA_HEADER,
            json=json_data)
print (response.json()['choices'][0]['text'])







```
print("Hello World")
```


```


In [4]:
json_data

{'model': 'vicuna-13b-v1.5-16k',
 'max_tokens': 256,
 'prompt': 'Write me a python script that print "Hello World"',
 'temperature': 0.0,
 'frequency_penalty': 0.0,
 'presence_penalty': 0.0}

In [25]:
from langchain.llms.base import LLM
from typing import Optional, List, Dict, Mapping, Any
import requests
from pydantic import Field

import pdb
# import os
# os.environ['CURL_CA_BUNDLE'] = ''

VICUNA_URL = 'http://10.100.100.104:8013/v1/completions'
VICUNA_HEADER = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

# messages_template = [
#     {
#       "content": "You are a helpful assistant.",
#       "role": "system"
#     },
#     {
#       "content": "What is the capital of France?",
#       "role": "user"
#     }
#   ]


class VicunaLLM(LLM):
    model_name: str = Field(None, alias='model_name')
    model_url: str = Field(None, alias='model_url')
    
    temperature: str = Field(None, alias='temperature')

    max_tokens: str = Field(None, alias='max_tokens')
    """The maximum number of tokens to generate in the completion.
    -1 returns as many tokens as possible given the prompt and
    the models maximal context size."""
    frequency_penalty: float = 0.0
    """Penalizes repeated tokens according to frequency."""
    presence_penalty: float = 0.0
    """Penalizes repeated tokens."""

    @property
    def _llm_type(self) -> str:
        return "vicuna"
    
    def __init__(
            self,
            model_name: str = "vicuna-13b-v1.5-16k",
            model_url: str = VICUNA_URL,
            max_tokens: int = 256,
            temperature: int = 0.0):
        super().__init__()
        self.model_name = model_name
        self.model_url  = model_url
        """What sampling temperature to use."""
        self.max_tokens = max_tokens
        self.temperature = temperature
    @property
    def _get_model_default_parameters(self):
        return {
            "model": self.model_name,
            "max_tokens": self.max_tokens,
            "temperature": self.temperature,
            "frequency_penalty": self.frequency_penalty,
            "presence_penalty": self.presence_penalty,
        }
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """
        Get all the identifying parameters
        """
        return {
            # 'model_path' : self.model_folder_path,
            'model_parameters': self._get_model_default_parameters
        }

    
    def _call(self, prompt: str, suffix: str = "", stop: Optional[List[str]] = ["Observation:"]) -> str:
        json_data = {
            "model": self.model_name,
            "max_tokens": self.max_tokens,
            'prompt': prompt,
            'temperature': self.temperature,
            "frequency_penalty": self.frequency_penalty,
            "presence_penalty": self.presence_penalty,
        }
        print(json_data)
        response = requests.post(
            self.model_url,
            headers=VICUNA_HEADER,
            json=json_data)
        # response.raise_for_status()
        print(response.json())
        print(response.json()['choices'][0]['text'])
        return response.json()['choices'][0]['text']


In [26]:
llm = VicunaLLM()

In [27]:
result = llm.generate(prompts=[prompt])

{'model': 'vicuna-13b-v1.5-16k', 'max_tokens': 256, 'prompt': 'Write me a python script that print "Hello World"', 'temperature': 0.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0}
{'id': 'cmpl-3h5pqHfTBCNvMUEqsWk6Er', 'object': 'text_completion', 'created': 1698135969, 'model': 'vicuna-13b-v1.5-16k', 'choices': [{'index': 0, 'text': '\n\n\n\n\n\n```\nprint("Hello World")\n```\n\n\n```', 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 12, 'total_tokens': 31, 'completion_tokens': 19}}






```
print("Hello World")
```


```


In [3]:
llm.dict()

{'model_parameters': {'model': 'vicuna-13b-v1.5-16k',
  'max_tokens': 256,
  'temperature': 0.0,
  'frequency_penalty': 0.0,
  'presence_penalty': 0.0},
 '_type': 'vicuna'}

In [10]:
import langchain
def get_prompts(
    params: Dict[str, Any], prompts: List[str]
):
    """Get prompts that are already cached."""
    llm_string = str(sorted([(k, v) for k, v in params.items()]))
    missing_prompts = []
    missing_prompt_idxs = []
    existing_prompts = {}
    for i, prompt in enumerate(prompts):
        if langchain.llm_cache is not None:
            cache_val = langchain.llm_cache.lookup(prompt, llm_string)
            if isinstance(cache_val, list):
                existing_prompts[i] = cache_val
            else:
                missing_prompts.append(prompt)
                missing_prompt_idxs.append(i)
    return existing_prompts, llm_string, missing_prompt_idxs, missing_prompts

In [11]:
params = llm.dict()
(
    existing_prompts,
    llm_string,
    missing_prompt_idxs,
    missing_prompts,
) = get_prompts(params, prompt)

In [15]:
missing_prompts

[]

In [17]:
existing_prompts

{}